# Classifying simulated events using a Convolutional Neural Network

In [1]:
# Imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from helper_functions import import_data, normalize_image_data
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
# Load images and labels.
DATA_PATH = "../data/"

images = np.load(DATA_PATH+"images_training.npy")
labels = np.load(DATA_PATH+"labels_training.npy")

# Split the training indices into training and validation. 
# Validate with 25% of the data (default). Can be adjusted.
x_idx = np.arange(images.shape[0])
train_idx, val_idx, not_used1, not_used2 = train_test_split(x_idx, x_idx, test_size = 0.25)

# Model
Now, you can build your own network from scratch, and that's a useful exercise. We're going to skip that
here, and use one of the popular, exisiting frameworks that are widely used in current research.
The most used base frameworks are [TensorFlow](https://www.tensorflow.org/), [PyTorch](https://pytorch.org/), and [Keras](https://keras.io/). Keras is a high-level API that abstracts a large amount of the process of building,
training, and testing a model. You will need either TensorFlow or PyTorch, and the Keras API will automatically
detect which base framework you have.

In this example we use TensorFlow (TF). TF also incorporates the keras API in it's own package,
which we will use.

## Build and compile

In [19]:
# Import the necessary TF models and layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [40]:
# Instantiate the Sequential model, and add layers to it.
model = Sequential()

# The model we build here is the one that has currently performed best when classifying the detector images.
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(16, 16, 1)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(2, activation='softmax'))

In [41]:
# Once the model is built, we need to compile it. This is where we specify the loss function,
# optimizer, and any metrics we need, even custom ones.

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Training
This is the point where we normalize our data, just as we pass it to the training function of the model.
The training run will display the progress as it goes through each batch.
$$ \text{num_batches} = \frac{\text{num_samples}}{\text{batch_size}}$$

In [42]:
# Set parameters for the training.
batch_size = 32
epochs = 20

In [43]:
# Setting validation data requires a tuple (val_input, val_targets). You can also just pass the
# entire training set without splitting, and specify validation_split instead of validation_data.
# The the model handles the splitting.

val_data = (normalize_image_data(images[val_idx]), labels[val_idx])
model.fit(
    x=normalize_image_data(images[train_idx]),
    y=labels[train_idx],
    validation_data=val_data,
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/10
4688/4688 [==============================] - 42s 9ms/step - loss: 0.6931 - accuracy: 0.4986 - val_loss: 0.6932 - val_accuracy: 0.5053
Epoch 2/10
4688/4688 [==============================] - 42s 9ms/step - loss: 0.6931 - accuracy: 0.5001 - val_loss: 0.6932 - val_accuracy: 0.4991
Epoch 3/10
4688/4688 [==============================] - 42s 9ms/step - loss: 0.6931 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.4977
Epoch 4/10
4688/4688 [==============================] - 44s 9ms/step - loss: 0.6931 - accuracy: 0.5013 - val_loss: 0.6932 - val_accuracy: 0.5021
Epoch 5/10
4688/4688 [==============================] - 42s 9ms/step - loss: 0.6931 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 6/10
4688/4688 [==============================] - 43s 9ms/step - loss: 0.6931 - accuracy: 0.4996 - val_loss: 0.6932 - val_accuracy: 0.4971
Epoch 7/10
 709/4688 [===>..........................] - ETA: 32s - loss: 0.6931 - accuracy: 0.5020

KeyboardInterrupt: 

In [ ]:



y_true = y_test.argmax(axis=-1)
tmp_predicted = loaded_model.predict(x_test)
y_pred = tmp_predicted.argmax(axis=-1)

# indices, relative distances and relative energies for test set
single_indices, double_indices, close_indices = event_indices(test_positions)
rel_distance_test = relative_distance(test_positions)
energy_diff_test = energy_difference(test_energies)
rel_energy_test = relative_energy(test_energies, noscale=True)


## Some numbers from results
Mean separation distances, mean relative energies, events with separation < 3mm etc.

In [ ]:
# Separate correct and wrong classifications
correct_doubles = np.where(y_pred[double_indices] == 1)[0]
wrong_doubles = np.where(y_pred[double_indices] == 0)[0]
correct_close_doubles = np.where(y_pred[close_indices] == 1)[0]

# Mean distances
mean_dist_correct = np.mean(rel_distance_test[double_indices][correct_doubles])
mean_dist_wrong = np.mean(rel_distance_test[double_indices][wrong_doubles])
mean_dist_all = np.mean(rel_distance_test[double_indices])

# Mean energies
mean_energy_all = np.mean(rel_energy_test[double_indices])
mean_energy_correct = np.mean(rel_energy_test[double_indices][correct_doubles])
mean_energy_wrong = np.mean(rel_energy_test[double_indices][wrong_doubles])


# Ratios
ratio_doubles = len(correct_doubles) / len(double_indices)
ratio_close = len(correct_close_doubles) / len(close_indices)


# Output
print("Number of events:")
print("Single: {} \nDouble: {} \nClose: {}\n".format(
    len(single_indices),
    len(double_indices),
    len(close_indices))
     )
print("Mean separations for classified double events:")
print("All doubles: {:.2f}mm \nCorrect: {:.2f}mm \nWrong: {:.2f}mm\n".format(
    mean_dist_all,
    mean_dist_correct,
    mean_dist_wrong)
     )

print("Mean relative energy for classified double events:")
print("All doubles: {:.2f} \nCorrect: {:.2f} \nWrong: {:.2f}\n".format(
    mean_energy_all,
    mean_energy_correct,
    mean_energy_wrong)
     )
print("Ratios of correctly classified double events:")
print("All doubles: {:.3f} \nClose: {:.3f}\n".format(
    ratio_doubles,
    ratio_close)
     )

print("Ratio of correct singles: %.3f" % (19749/len(single_indices)))

# Results on test set

## Confusion Matrix
The confusion matrix can be a useful metric to gain a little bit
more insight into specifically what the model gets wrong.
* Top left: Single events classified as single events
* Top right: Single events classified as double events
* Bottom left: Double events classified as single events
* Bottom right: Double events classified as double events

In [ ]:
from analysis_functions.plotting import plot_confusion_matrix
classes = ["single", "double"]
title = net +" Confusion Matrix"
plot_confusion_matrix(y_true, y_pred, classes, title=title)
plt.show()
#plt.savefig(FIGURE_PATH+net+"_confmat.pdf", format="pdf")

## F1 Score

In [ ]:
from sklearn.metrics import f1_score
score = f1_score(y_true, y_pred)
print("F1-score: ", score)

### ROC curves
#### All events

In [ ]:
from analysis_functions.plotting import plot_roc_curve
from sklearn.metrics import roc_auc_score

plot_roc_curve(y_true, tmp_predicted[:,1])
plt.show()


print("Area under curve:",roc_auc_score(y_true, tmp_predicted[:,1]))

#### Close events

In [ ]:
single_close_true = np.concatenate((y_true[single_indices], y_true[close_indices]), axis=0)
single_close_pred = np.concatenate((tmp_predicted[single_indices,1], tmp_predicted[close_indices,1]), axis=0)
plot_roc_curve(single_close_true, single_close_pred)
plt.show()
print("Area under curve:",roc_auc_score(single_close_true, single_close_pred))
single_close_pred = np.concatenate((y_pred[single_indices], y_pred[close_indices]), axis=0)
score_close = f1_score(single_close_true, single_close_pred)
print("F1-score close: ", score_close)


# Distributions and scatterplot

## Test set

### Comparing correct and misclassified double events

In [ ]:
dist_bins = np.arange(0, np.amax(rel_distance_test), 0.5)
energy_bins = np.arange(0, np.amax(energy_diff_test), 0.02)
fig, ax = plt.subplots(1, 2, figsize=(12,4))
ax[0].hist(rel_distance_test[double_indices][correct_doubles], bins=dist_bins, alpha=0.5, label="correct")
ax[0].hist(rel_distance_test[double_indices][wrong_doubles], bins=dist_bins, alpha=0.5, label="wrong")
ax[0].set_title("Distribution of separation distances\n for classified double events")
ax[0].set_xlabel("Separation distance [mm]")
ax[0].set_ylabel("Number of events")
ax[0].legend()
ax[1].hist(rel_energy_test[double_indices][correct_doubles], bins=energy_bins, alpha=0.5, label="correct")
ax[1].hist(rel_energy_test[double_indices][wrong_doubles], bins=energy_bins, alpha=0.5, label="wrong")
ax[1].set_title("Distribution of relative energy \n for classified double events")
ax[1].set_xlabel("Relative energy")
ax[1].set_ylabel("Number of events")
ax[1].legend()
fig.savefig(FIGURE_PATH+net+"_relative_test_compare.pdf", format="pdf")

In [ ]:
dist_bins = np.arange(0, np.amax(rel_distance_test), 0.5)
energy_bins = np.arange(0, 10, 0.1)
fig, ax = plt.subplots(1, 2, figsize=(12,4))
#ax[0].hist(rel_distance_test[double_indices][correct_doubles], bins=dist_bins, alpha=0.5, label="correct")
ax[0].hist(rel_distance_test[double_indices][wrong_doubles], bins=dist_bins, alpha=0.5, label="wrong")
ax[0].set_title("Distribution of Relative distances\n for classified double events")
ax[0].set_xlabel("Relative distance [mm]")
ax[0].set_ylabel("Number of events")
ax[0].legend()
#ax[1].hist(rel_energy_test[double_indices][correct_doubles], bins=energy_bins, alpha=0.5, label="correct")
#ax[1].hist(rel_energy_test[double_indices][wrong_doubles], bins=energy_bins, alpha=0.5, label="wrong")
ax[1].hist(rel_energy_test[double_indices][wrong_doubles], label="wrong")
ax[1].set_title("Distribution of relative energy \n for classified double events")
ax[1].set_xlabel("Relative energy")
ax[1].set_ylabel("Number of events")
ax[1].legend()
fig.savefig(FIGURE_PATH+net+"_relative_test_compare.pdf", format="pdf")

### Scatterplot relative distance vs. relative energy

In [ ]:
plt.scatter(
    rel_distance_test[double_indices][wrong_doubles], 
    rel_energy_test[double_indices][wrong_doubles],
    marker='.',
    )
plt.title("Separation distance vs. relative energy for misclassified double events")
plt.xlabel("Separation distance [mm]")
plt.ylabel("Relative energy")
plt.show()

## Events with high separation distance
and varying relative energy above a threshold

In [ ]:
# Grab the indices
separation_lim = 20.0
energy_lim = 0.2
high_relD = np.where(rel_distance_test[double_indices][wrong_doubles] > separation_lim)[0]
high_relE = np.where(rel_energy_test[double_indices][wrong_doubles] > energy_lim)[0]

# Then get the overlapping indices
high_both = np.array(list(set(high_relD).intersection(set(high_relE))))
print("Found {} events.".format(len(high_both)))



In [ ]:
# Use the indices to fetch the images and other statistics we want.
images_plot_high = images[test_idx][double_indices][wrong_doubles][high_both][:,:,:,0]
rel_pos_plot_high = rel_distance_test[double_indices][wrong_doubles][high_both]
rel_energy_plot_high = rel_energy_test[double_indices][wrong_doubles][high_both]
energy_plot_high = energies[test_idx][double_indices][wrong_doubles][high_both]

# Plot the events images with relative separation, energies, and relative energy
# to the top left of each image
fig, ax = plt.subplots(3, 2, figsize=(12,12))
for i in range(3):
    for j in range(2):
        if i*2+j >= len(high_both):
            fig.delaxes(ax.flatten()[i*2 + j])
            continue
        ax[i, j].imshow(images_plot_high[i*2 + j])
        rel_pos = rel_pos_plot_high[i*2 + j]
        rel_E = rel_energy_plot_high[i*2 + j]
        E1 = energy_plot_high[i*2 + j, 0]
        E2 = energy_plot_high[i*2 + j, 1]
        relp = "dist = {:.2f}mm".format(rel_pos[0])
        rele = "rel_E = {:.2f}".format(rel_E[0])
        e1_txt = "E1 = {:.2f} MeV".format(E1)
        e2_txt = "E2 = {:.2f} MeV".format(E2)
        ax[i, j].text(-10, 0, relp, fontsize=11)
        ax[i, j].text(-10, 1, e1_txt, fontsize=11)
        ax[i, j].text(-10, 2, e2_txt, fontsize=11)
        ax[i, j].text(-10, 3, rele, fontsize=11)
        
fig.suptitle("Misclassified events with large separation distance", fontsize=16)
#fig.savefig(FIGURE_PATH+net+"_misclassified_large_dist.pdf", format="pdf")
fig.show()

# Plots of events that no networks were able to classify correctly

In [ ]:
# Load indices
OUTPUT_PATH = MODEL_PATH = "../../data/output/"
fname_indices = "never_correct_indices_rerun.txt"
never_correct = np.loadtxt(OUTPUT_PATH + fname_indices, dtype=int).tolist()

rel_distance_all = relative_distance(positions)
rel_energy_all = relative_energy(energies)



## Distributions for relative distance and relative energy

In [ ]:
# Calculate bins
dist_bins = np.arange(0, np.amax(rel_distance_all), 0.5)
energy_bins = np.arange(0, np.amax(rel_energy_all), 0.02)

fig, ax = plt.subplots(1, 2, figsize=(12,4))
ax[0].hist(rel_distance_all[never_correct], bins=dist_bins)
ax[0].set_title("Distribution of relative distance\n for always misclassified events")
ax[0].set_xlabel("Relative distance [mm]")
ax[0].set_ylabel("Number of events")
ax[0].legend()

ax[1].hist(rel_energy_all[never_correct], bins=energy_bins)
ax[1].set_title("Distribution of relative energy\n for always misclassified events")
ax[1].set_xlabel("Relative energy")
ax[1].set_ylabel("Number of events")
ax[1].legend()

#fig.savefig(FIGURE_PATH+net+"_relative_noncorrect.pdf", format="pdf")
fig.show()

## Plot images of some of the events

In [ ]:
print("Number of always misclassified events:", len(never_correct))

images_plot = images[never_correct][:,:,:,0]
rel_dist_plot = rel_distance_all[never_correct]
rel_energy_plot = rel_energy_all[never_correct]
energy_plot = energies[never_correct]
fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,12))
index = 10
for i in range(3):
    for j in range(3):
        ax[i, j].imshow(images_plot[index + i*3 + j])
        rel_dist = rel_dist_plot[index + i*3 + j]
        rel_energy = rel_energy_plot[index + i*3 + j]
        E1 = energy_plot[i*3 + j, 0]
        E2 = energy_plot[i*3 + j, 1]
        rel_d = "rel_dist = {:.2f}mm".format(rel_dist[0])
        rel_e = "rel_E = {:.2f}".format(rel_energy[0])

        e1_txt = "E1 = {:.2f} MeV".format(E1)
        e2_txt = "E2 = {:.2f} MeV".format(E2)
        ax[i, j].text(0,-2, rel_d, color='black', fontsize=11)
        ax[i, j].text(0,-1, rel_e, color='black', fontsize=11)
        ax[i, j].text(0,1, e1_txt, color='cyan', fontsize=11)
        ax[i, j].text(0,2, e2_txt, color='cyan', fontsize=11)


fig.suptitle("Images of always misclassified double events", fontsize=16)
#fig.savefig(FIGURE_PATH+net+"_nocorrect_samples.pdf", format="pdf")

fig.show()
        

        

In [ ]:
# Plot some images, with electron origin positions
%matplotlib inline

images = images.reshape(images.shape[0],16,16)

fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,12))
for i in range(3):
    for j in range(3):
        # plot image
        ax[i, j].imshow(images[index + i*3 + j])
        
        # plot origin of event
        x = positions[index + i*3 + j, 0]
        y = positions[index + i*3 + j, 1]
        ax[i, j].plot(x, y, 'rx')
        ax[i, j].set_title('single')
        if positions[index + i*3 + j, 3] != -100:
            x2 = positions[index + i*3 + j, 2]
            y2 = positions[index + i*3 + j, 3]
            ax[i, j].plot(x2, y2, 'rx')
            ax[i, j].set_title('double')
        
plt.show()

# Distribution of position around highest intensity pixel
In previous work data analysis showed that most event positions are within the highest intensity pixel,
and all (verify!) events are within the two highest intensity pixels,
It might be reasonable to look at how the predicted positions are distributed around the highest intensity
pixel.

In [ ]:
imgs = images[single_indices].reshape(images[single_indices].shape[0],16,16)

# get index of highest energy pixel
print(np.unravel_index(np.argmax(imgs[0], axis=None), imgs[0].shape))
fix, ax = plt.subplots()
ax.imshow(imgs[0])
ax.plot(0,0, 'rx')

In [ ]:
config = {
    "DATA_PATH": "../../data/real/anodedata.txt",              
    "MODEL_PATH": "../../data/output/models/",                
    "CLASSIFIER": "Project-0.97.hdf5",                      
    "SINGLE_ENERGY_MODEL": "single_energy_model_name.hdf5",    
    "SINGLE_POSITION_MODEL": "single_position_model_name.hdf5",
    "DOUBLE_ENERGY_MODEL": "double_energy_model_name.hdf5",    
    "DOUBLE_POSITION_MODEL": "double_position_model_name.hdf5" 
}

data = import_real_data(config)
print(data['image'].type)
